### LSTM 을 활용한 주가 예측 모델
##### 주가 보조지표 추가

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import datetime as dt


%matplotlib inline
warnings.filterwarnings('ignore')

sqlite DB에서 데이터 불러오기

In [32]:
import sqlite3
con = sqlite3.connect(r"C:\Users\구남이\OneDrive\바탕 화면\활동들\한이음 멘토링\stock.db")
df = pd.read_sql("SELECT * FROM '039490'", con, index_col=None)
df.rename(columns={'index':'date'}, inplace=True)
df = df.set_index('date')
df.index = pd.DatetimeIndex(df.index)
df.sort_index(ascending=True, inplace=True)
df

,open,high,low,close,volume
date,,,,,
2004-04-23,6076,6799,5353,5353,1415109
2004-04-26,5436,5741,5261,5398,343283
2004-04-27,5452,5482,5033,5063,299489
2004-04-28,5071,5452,4995,5444,182604
2004-04-29,5117,5436,5025,5101,109911
...,...,...,...,...,...
2022-05-11,87500,87500,85500,85800,108071
2022-05-12,84700,85300,82700,83200,104007
2022-05-13,84000,87400,83500,85800,86525


In [33]:
# 이동평균선 추가
ma = [5,20,60,120]
for days in ma:
    df['ma_'+str(days)] = df['close'].rolling(window = days).mean()
df.dropna(inplace=True)
df 

,open,high,low,close,volume,ma_5,ma_20,ma_60,ma_120
date,,,,,,,,,
2004-10-14,3959,4066,3906,4035,171655,3940.80000,3963.60000,3421.08333,3346.36667
2004-10-15,4066,4096,3959,3990,128530,3959.20000,3946.85000,3442.28333,3335.00833
2004-10-18,3997,3997,3860,3868,77233,3944.00000,3935.05000,3461.38333,3322.25833
2004-10-19,3868,3914,3830,3853,63272,3935.00000,3922.90000,3481.18333,3312.17500
2004-10-20,3807,3853,3769,3769,118227,3903.00000,3909.60000,3499.45000,3298.21667
...,...,...,...,...,...,...,...,...,...
2022-05-11,87500,87500,85500,85800,108071,89460.00000,94620.00000,97348.33333,100015.83333
2022-05-12,84700,85300,82700,83200,104007,87480.00000,93785.00000,97120.00000,99788.33333
2022-05-13,84000,87400,83500,85800,86525,86360.00000,93050.00000,96945.00000,99603.33333


In [34]:
! pip install ta
import ta

H, L, C, V = df['high'], df['low'], df['close'], df['volume']

# df['bol_high'] = ta.volatility.bollinger_hband(C)
# df['bol_low']  = ta.volatility.bollinger_lband(C)
df['MFI'] = ta.volume.money_flow_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['ADI'] = ta.volume.acc_dist_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['OBV'] = ta.volume.on_balance_volume(close=C, volume=V, fillna=True)
df['CMF'] = ta.volume.chaikin_money_flow(
    high=H, low=L, close=C, volume=V, fillna=True)

df['FI'] = ta.volume.force_index(close=C, volume=V, fillna=True)
df['EOM, EMV'] = ta.volume.ease_of_movement(
    high=H, low=L, volume=V, fillna=True)

df['VPT'] = ta.volume.volume_price_trend(close=C, volume=V, fillna=True)
df['NVI'] = ta.volume.negative_volume_index(close=C, volume=V, fillna=True)
df['VMAP'] = ta.volume.volume_weighted_average_price(
    high=H, low=L, close=C, volume=V, fillna=True)

# Volatility
df['ATR'] = ta.volatility.average_true_range(
    high=H, low=L, close=C, fillna=True)
df['BHB'] = ta.volatility.bollinger_hband(close=C, fillna=True)
df['BLB'] = ta.volatility.bollinger_lband(close=C, fillna=True)
df['KCH'] = ta.volatility.keltner_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['KCL'] = ta.volatility.keltner_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['KCM'] = ta.volatility.keltner_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['DCH'] = ta.volatility.donchian_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['DCL'] = ta.volatility.donchian_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['DCM'] = ta.volatility.donchian_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['UI'] = ta.volatility.ulcer_index(close=C, fillna=True)
# Trend
df['SMA'] = ta.trend.sma_indicator(close=C, fillna=True)
df['EMA'] = ta.trend.ema_indicator(close=C, fillna=True)
df['WMA'] = ta.trend.wma_indicator(close=C, fillna=True)
df['MACD'] = ta.trend.macd(close=C, fillna=True)
df['ADX'] = ta.trend.adx(high=H, low=L, close=C, fillna=True)
df['-VI'] = ta.trend.vortex_indicator_neg(
    high=H, low=L, close=C, fillna=True)
df['+VI'] = ta.trend.vortex_indicator_pos(
    high=H, low=L, close=C, fillna=True)
df['TRIX'] = ta.trend.trix(close=C, fillna=True)
df['MI'] = ta.trend.mass_index(high=H, low=L, fillna=True)
df['CCI'] = ta.trend.cci(high=H, low=L, close=C, fillna=True)
df['DPO'] = ta.trend.dpo(close=C, fillna=True)
df['KST'] = ta.trend.kst(close=C, fillna=True)
df['Ichimoku'] = ta.trend.ichimoku_a(high=H, low=L, fillna=True)
df['Parabolic SAR'] = ta.trend.psar_down(
    high=H, low=L, close=C, fillna=True)
df['STC'] = ta.trend.stc(close=C, fillna=True)
# Momentum
df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
df['SRSI'] = ta.momentum.stochrsi(close=C, fillna=True)
df['TSI'] = ta.momentum.tsi(close=C, fillna=True)
df['UO'] = ta.momentum.ultimate_oscillator(
    high=H, low=L, close=C, fillna=True)
df['SR'] = ta.momentum.stoch(close=C, high=H, low=L, fillna=True)
df['WR'] = ta.momentum.williams_r(high=H, low=L, close=C, fillna=True)
df['AO'] = ta.momentum.awesome_oscillator(high=H, low=L, fillna=True)
df['KAMA'] = ta.momentum.kama(close=C, fillna=True)
df['ROC'] = ta.momentum.roc(close=C, fillna=True)
df['PPO'] = ta.momentum.ppo(close=C, fillna=True)
df['PVO'] = ta.momentum.pvo(volume=V, fillna=True)

df


,open,high,low,close,volume,ma_5,ma_20,ma_60,ma_120,MFI,ADI,OBV,CMF,FI,"EOM, EMV",VPT,NVI,VMAP,ATR,BHB,BLB,KCH,KCL,KCM,DCH,DCL,DCM,UI,SMA,EMA,WMA,MACD,ADX,-VI,+VI,TRIX,MI,CCI,DPO,KST,Ichimoku,Parabolic SAR,STC,RSI,SRSI,TSI,UO,SR,WR,AO,KAMA,ROC,PPO,PVO
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-10-14,3959,4066,3906,4035,171655,3940.80000,3963.60000,3421.08333,3346.36667,50.00000,105138.68750,171655,0.61250,0.00000,0.00000,-159682.99753,1000.00000,4002.33333,0.00000,4035.00000,4035.00000,4162.33333,3842.33333,4002.33333,4066.00000,3906.00000,3986.00000,0.00000,4035.00000,4035.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-93.38108,1.00000,0.00000,57355.97381,-934.27373,3986.00000,4066.00000,0.00000,100.00000,0.00000,0.00000,0.00000,80.62500,-19.37500,0.00000,4035.00000,0.00000,0.00000,0.00000
2004-10-15,4066,4096,3959,3990,128530,3959.20000,3946.85000,3442.28333,3335.00833,100.00000,34775.54881,43125,0.11585,-5783850.00000,4423480.89940,-161806.17641,988.84758,4007.75681,0.00000,4057.50000,3967.50000,4157.16667,3860.16667,4008.66667,4096.00000,3906.00000,4001.00000,0.00000,4012.50000,4028.07692,0.00000,-3.58974,0.00000,0.00186,0.00330,-0.00218,1.97687,66.66667,57378.47381,-934.64023,4001.00000,4066.00000,0.00000,0.00000,0.00000,-100.00000,10.43771,44.21053,-55.78947,0.00000,4012.55277,0.00000,-0.08904,-2.04212
2004-10-18,3997,3997,3860,3868,77233,3944.00000,3935.05000,3461.38333,3322.25833,63.09424,-33437.53878,-34108,-0.08860,-6303646.57143,-17561146.14219,-3794.93035,958.61214,3987.41127,0.00000,4105.43692,3823.22975,4119.88889,3830.55556,3975.22222,4096.00000,3860.00000,3978.00000,0.00000,3964.33333,4003.44970,0.00000,-16.09351,0.00000,0.00594,0.00395,-0.01162,2.93951,-100.00000,57426.64047,-935.42482,3978.00000,4066.00000,0.00000,0.00000,0.00000,-100.00000,8.98618,3.38983,-96.61017,0.00000,3942.93216,0.00000,-0.40038,-6.30064
2004-10-19,3868,3914,3830,3853,63272,3935.00000,3922.90000,3481.18333,3312.17500,48.56973,-62060.58640,-97380,-0.14083,-5538708.48980,-7500948.28676,-2606.87739,954.89467,3969.93181,0.00000,4092.15667,3780.84333,4077.33333,3818.33333,3947.83333,4096.00000,3830.00000,3963.00000,0.00000,3936.50000,3980.30360,0.00000,-26.90308,0.00000,0.00882,0.00488,-0.02624,3.83838,-90.04566,57454.47381,-935.87820,3963.00000,4096.00000,0.00000,0.00000,0.00000,-100.00000,11.96911,8.64662,-91.35338,0.00000,3901.74617,0.00000,-0.67137,-10.66818
2004-10-20,3807,3853,3769,3769,118227,3903.00000,3909.60000,3499.45000,3298.21667,34.14378,-180287.58640,-215607,-0.32257,-6166188.41983,-4334035.37263,-2822.85686,954.89467,3933.35176,0.00000,4096.23354,3709.76646,4038.06667,3797.26667,3917.66667,4096.00000,3769.00000,3932.50000,0.00000,3903.00000,3947.79535,0.00000,-41.76639,0.00000,0.01131,0.00527,-0.04771,4.69621,-110.50061,57487.97381,-936.42388,3932.50000,4085.36000,0.00000,0.00000,0.00000,-100.00000,10.29900,0.00000,-100.00000,0.00000,3842.23824,0.00000,-1.04689,-11.08878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-11,87500,87500,85500,85800,108071,89460.00000,94620.00000,97348.33333,100015.83333,20.75239,11676203.01587,49229575,-0.04830,-79930656.17839,-2405825799.70575,-3829.30309,3.40831,92053.42607,2080.61688,102228.83697,87011.16303,96403.33333,92843.33333,94623.33333,101000.00000,85500.00000,90500.00000,7.31728,92191.66667,91652.50973,90017.77778,-2556.81951,24.01638,1.23649,0.56081,-0.23399,24.86296,-189.70866,380.00000,-59.51709,91875.00000,92289.50506,0.00038,21.02872,0.00000,-35.84006,35.39960,2.17391,-97.82609,-6237.35294,90199.19667,-11.18012,-2.71398,13.36054
2022-05-12,84700,85300,82700,83200,104007,87480.00000,93785.00000,97120.00000,99788.33333,12.83842,11612198.70818,49125568,-0.14207,-107143162.43862,-6249579355.23571,-5853.50227,3.30503,90591.28878,2153.42996,102480.57934,85089.42066,95675.00000,92005.00000

In [35]:
# 환율 데이터
! pip install Finance-DataReader
import FinanceDataReader as fdr

start_date = str(df.index[0])[0:10]
end_date = str(df.index[-1])[0:10]
rate_df = pd.DataFrame()
rate_df['USD/KRW_change'] = fdr.DataReader('USD/KRW', start=start_date, end=end_date)['Change']
rate_df['USD/EUR_change'] = fdr.DataReader('USD/EUR', start=start_date, end=end_date)['Change']
rate_df['USD/JPY_change'] = fdr.DataReader('USD/JPY', start=start_date, end=end_date)['Change']
rate_df['EUR/USD_change'] = fdr.DataReader('EUR/USD', start=start_date, end=end_date)['Change']
rate_df['CNY/KRW_change'] = fdr.DataReader('CNY/KRW', start=start_date, end=end_date)['Change']
rate_df['JPY/KRW_change'] = fdr.DataReader('JPY/KRW', start=start_date, end=end_date)['Change']
rate_df['AUD/USD_change'] = fdr.DataReader('AUD/USD', start=start_date, end=end_date)['Change']
rate_df['EUR/JPY_change'] = fdr.DataReader('EUR/JPY', start=start_date, end=end_date)['Change']
rate_df['USD/RUB_change'] = fdr.DataReader('USD/RUB', start=start_date, end=end_date)['Change']

rate_df

,USD/KRW_change,USD/EUR_change,USD/JPY_change,EUR/USD_change,CNY/KRW_change,JPY/KRW_change,AUD/USD_change,EUR/JPY_change,USD/RUB_change
Date,,,,,,,,,
2004-10-14,-0.00350,-0.00280,-0.00130,0.00280,-0.00350,-0.00230,0.00430,0.00150,-0.00010
2004-10-15,0.00090,-0.00720,-0.00290,0.00690,0.00090,0.00380,0.00080,0.00410,0.00000
2004-10-18,-0.00350,-0.00200,-0.00080,0.00240,-0.00350,-0.00260,-0.00550,0.00160,0.00020
2004-10-19,-0.00090,-0.00170,-0.00870,0.00180,-0.00090,0.00790,0.00230,-0.00730,0.00000
2004-10-20,0.00090,-0.00540,-0.00010,0.00540,0.00090,0.00090,0.01090,0.00550,-0.00010
...,...,...,...,...,...,...,...,...,...
2022-05-11,0.00200,0.00150,-0.00360,-0.00150,0.00370,0.00560,0.00030,-0.00510,-0.03170
2022-05-12,0.00910,0.01290,-0.01260,-0.01260,-0.00040,0.02200,-0.01200,-0.02500,-0.05800
2022-05-13,-0.00950,-0.00320,0.00680,0.00310,-0.00980,-0.01620,0.01240,0.00990,0.01970


In [36]:
from sklearn.preprocessing import MinMaxScaler
# 환율 변수를 제외한 피처값 스케일링
df_feature = df.drop('close', axis=1)
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df_feature)
scaled_df = pd.DataFrame(scaled_df, columns = df_feature.columns, index = df.index)

scaled_df

,open,high,low,volume,ma_5,ma_20,ma_60,ma_120,MFI,ADI,OBV,CMF,FI,"EOM, EMV",VPT,NVI,VMAP,ATR,BHB,BLB,KCH,KCL,KCM,DCH,DCL,DCM,UI,SMA,EMA,WMA,MACD,ADX,-VI,+VI,TRIX,MI,CCI,DPO,KST,Ichimoku,Parabolic SAR,STC,RSI,SRSI,TSI,UO,SR,WR,AO,KAMA,ROC,PPO,PVO
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-10-14,0.00378,0.00402,0.00364,0.11962,0.00341,0.00343,0.00000,0.00086,0.50000,0.29452,0.01255,0.90366,0.34291,0.37701,0.00687,0.87772,0.00370,0.00000,0.00293,0.00526,0.00407,0.00325,0.00369,0.00286,0.00410,0.00361,0.00000,0.00401,0.00399,0.00000,0.44345,0.00000,0.00000,0.00000,0.00000,0.00000,0.42503,0.99587,0.00286,0.00342,0.00323,0.00000,1.00000,0.00000,0.56549,0.00000,0.80625,0.80625,0.48142,0.00362,0.33736,0.44125,0.38169
2004-10-15,0.00445,0.00420,0.00399,0.08907,0.00353,0.00331,0.00015,0.00077,1.00000,0.29108,0.01002,0.52387,0.34140,0.37703,0.00000,0.86790,0.00374,0.00000,0.00307,0.00477,0.00403,0.00337,0.00373,0.00304,0.00410,0.00371,0.00000,0.00386,0.00395,0.00000,0.44321,0.00000,0.00128,0.00230,0.97714,0.03373,0.50940,0.99619,0.00263,0.00352,0.00323,0.00000,0.00000,0.00000,0.00000,0.11736,0.44211,0.44211,0.48142,0.00347,0.33736,0.43810,0.36453
2004-10-18,0.00402,0.00360,0.00334,0.05273,0.00343,0.00323,0.00029,0.00068,0.63094,0.28775,0.00850,0.36753,0.34126,0.37694,0.51131,0.84127,0.00360,0.00000,0.00336,0.00373,0.00379,0.00316,0.00351,0.00304,0.00376,0.00356,0.00000,0.00353,0.00378,0.00000,0.44240,0.00000,0.00411,0.00275,0.97704,0.06697,0.29846,0.99687,0.00214,0.00337,0.00323,0.00000,0.00000,0.00000,0.00000,0.10104,0.03390,0.03390,0.48142,0.00299,0.33736,0.42712,0.32876
2004-10-19,0.00320,0.00309,0.00314,0.04284,0.00337,0.00315,0.00044,0.00060,0.48570,0.28635,0.00726,0.32759,0.34146,0.37698,0.51515,0.83800,0.00348,0.00000,0.00328,0.00342,0.00351,0.00307,0.00332,0.00304,0.00354,0.00346,0.00000,0.00334,0.00362,0.00000,0.44170,0.00000,0.00610,0.00340,0.97688,0.09800,0.31106,0.99726,0.00185,0.00327,0.00341,0.00000,0.00000,0.00000,0.00000,0.13458,0.08647,0.08647,0.48142,0.00271,0.33736,0.41756,0.29207
2004-10-20,0.00282,0.00272,0.00274,0.08177,0.00316,0.00306,0.00057,0.00049,0.34144,0.28058,0.00494,0.18862,0.34130,0.37699,0.51445,0.83800,0.00323,0.00000,0.00331,0.00291,0.00325,0.00292,0.00311,0.00304,0.00309,0.00326,0.00000,0.00312,0.00340,0.00000,0.44074,0.00000,0.00782,0.00367,0.97666,0.12762,0.28517,0.99773,0.00151,0.00307,0.00334,0.00000,0.00000,0.00000,0.00000,0.11580,0.00000,0.00000,0.48142,0.00230,0.33736,0.40431,0.28853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-11,0.52872,0.51247,0.53816,0.07458,0.56724,0.62613,0.68425,0.73431,0.20752,0.85970,0.97583,0.39834,0.32203,0.36732,0.51119,0.00009,0.60038,0.26200,0.60812,0.60663,0.60724,0.63812,0.62544,0.59427,0.60560,0.57728,0.26159,0.60469,0.60114,0.59214,0.27781,0.36167,0.85521,0.39073,0.97471,0.82394,0.18492,0.19253,0.55573,0.58875,0.54130,0.00000,0.21029,0.00000,0.36282,0.39804,0.02174,0.02174,0.33429,0.59224,0.23364,0.34548,0.49393
2022-05-12,0.51113,0.49906,0.51982,0.07170,0.55418,0.62039,0.68259,0.73259,0.12838,0.85658,0.97379,0.32664,0.31492,0.35184,0.50464,0.00000,0.59048,0.27117,0.60967,0.59271,0.60248,0.63214,0.62006,0.59427,0.58496,0.56800,0.29257,0.59799,0.59228,0.58126,0.24640,0.40378,0.93124,0.32332,0.97428,0.83082,0.16046,0.19302,0.54966,0.57943,0.53301,0.00000,0.17870,0.00000,0.33266,0.33748,0.03311,0.03311,0.29844,0.57821,0.22213,0.32633,0.52508
2022-05-13,0.50673,0.51186,0.52506,0.05931,0.54680,0.61535,0.68131,0.73118,0.21839,0.85734,0.97549,0.31588,0.32731,0.40333,0.52214,0.00009,0.58493,0.28958,0.60569,0.58673,0.59832,0.62625,0.61505,0.58695,0.58496,0.56800,0.30545,0.59322,0.58751,0.57538,0.23747,0.42330,0.84511,0.39813,0.97388,0.84055,0.23803,0.19916,0.54450,0.57910,0.52297,0.00000,0.29306,0.46565,0.33157,0.40061,0.23134,0.23134,0.27487,0.57598,0.25463,0.32039,0.52849


In [37]:
# 타켓 스케일링
scaler1 = MinMaxScaler()
scaled_df['close'] = scaler1.fit_transform(df['close'].values.reshape(-1,1))

In [38]:
pd.set_option('display.max_rows', 10) 
pd.set_option('display.max_columns', None) 

scaled_df = pd.merge(scaled_df, rate_df, left_index=True, right_index=True)

In [39]:
scaled_df

,open,high,low,volume,ma_5,ma_20,ma_60,ma_120,MFI,ADI,OBV,CMF,FI,"EOM, EMV",VPT,NVI,VMAP,ATR,BHB,BLB,KCH,KCL,KCM,DCH,DCL,DCM,UI,SMA,EMA,WMA,MACD,ADX,-VI,+VI,TRIX,MI,CCI,DPO,KST,Ichimoku,Parabolic SAR,STC,RSI,SRSI,TSI,UO,SR,WR,AO,KAMA,ROC,PPO,PVO,close,USD/KRW_change,USD/EUR_change,USD/JPY_change,EUR/USD_change,CNY/KRW_change,JPY/KRW_change,AUD/USD_change,EUR/JPY_change,USD/RUB_change
2004-10-14,0.00378,0.00402,0.00364,0.11962,0.00341,0.00343,0.00000,0.00086,0.50000,0.29452,0.01255,0.90366,0.34291,0.37701,0.00687,0.87772,0.00370,0.00000,0.00293,0.00526,0.00407,0.00325,0.00369,0.00286,0.00410,0.00361,0.00000,0.00401,0.00399,0.00000,0.44345,0.00000,0.00000,0.00000,0.00000,0.00000,0.42503,0.99587,0.00286,0.00342,0.00323,0.00000,1.00000,0.00000,0.56549,0.00000,0.80625,0.80625,0.48142,0.00362,0.33736,0.44125,0.38169,0.00417,-0.00350,-0.00280,-0.00130,0.00280,-0.00350,-0.00230,0.00430,0.00150,-0.00010
2004-10-15,0.00445,0.00420,0.00399,0.08907,0.00353,0.00331,0.00015,0.00077,1.00000,0.29108,0.01002,0.52387,0.34140,0.37703,0.00000,0.86790,0.00374,0.00000,0.00307,0.00477,0.00403,0.00337,0.00373,0.00304,0.00410,0.00371,0.00000,0.00386,0.00395,0.00000,0.44321,0.00000,0.00128,0.00230,0.97714,0.03373,0.50940,0.99619,0.00263,0.00352,0.00323,0.00000,0.00000,0.00000,0.00000,0.11736,0.44211,0.44211,0.48142,0.00347,0.33736,0.43810,0.36453,0.00389,0.00090,-0.00720,-0.00290,0.00690,0.00090,0.00380,0.00080,0.00410,0.00000
2004-10-18,0.00402,0.00360,0.00334,0.05273,0.00343,0.00323,0.00029,0.00068,0.63094,0.28775,0.00850,0.36753,0.34126,0.37694,0.51131,0.84127,0.00360,0.00000,0.00336,0.00373,0.00379,0.00316,0.00351,0.00304,0.00376,0.00356,0.00000,0.00353,0.00378,0.00000,0.44240,0.00000,0.00411,0.00275,0.97704,0.06697,0.29846,0.99687,0.00214,0.00337,0.00323,0.00000,0.00000,0.00000,0.00000,0.10104,0.03390,0.03390,0.48142,0.00299,0.33736,0.42712,0.32876,0.00312,-0.00350,-0.00200,-0.00080,0.00240,-0.00350,-0.00260,-0.00550,0.00160,0.00020
2004-10-19,0.00320,0.00309,0.00314,0.04284,0.00337,0.00315,0.00044,0.00060,0.48570,0.28635,0.00726,0.32759,0.34146,0.37698,0.51515,0.83800,0.00348,0.00000,0.00328,0.00342,0.00351,0.00307,0.00332,0.00304,0.00354,0.00346,0.00000,0.00334,0.00362,0.00000,0.44170,0.00000,0.00610,0.00340,0.97688,0.09800,0.31106,0.99726,0.00185,0.00327,0.00341,0.00000,0.00000,0.00000,0.00000,0.13458,0.08647,0.08647,0.48142,0.00271,0.33736,0.41756,0.29207,0.00303,-0.00090,-0.00170,-0.00870,0.00180,-0.00090,0.00790,0.00230,-0.00730,0.00000
2004-10-20,0.00282,0.00272,0.00274,0.08177,0.00316,0.00306,0.00057,0.00049,0.34144,0.28058,0.00494,0.18862,0.34130,0.37699,0.51445,0.83800,0.00323,0.00000,0.00331,0.00291,0.00325,0.00292,0.00311,0.00304,0.00309,0.00326,0.00000,0.00312,0.00340,0.00000,0.44074,0.00000,0.00782,0.00367,0.97666,0.12762,0.28517,0.99773,0.00151,0.00307,0.00334,0.00000,0.00000,0.00000,0.00000,0.11580,0.00000,0.00000,0.48142,0.00230,0.33736,0.40431,0.28853,0.00250,0.00090,-0.00540,-0.00010,0.00540,0.00090,0.00090,0.01090,0.00550,-0.00010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-11,0.52872,0.51247,0.53816,0.07458,0.56724,0.62613,0.68425,0.73431,0.20752,0.85970,0.97583,0.39834,0.32203,0.36732,0.51119,0.00009,0.60038,0.26200,0.60812,0.60663,0.60724,0.63812,0.62544,0.59427,0.60560,0.57728,0.26159,0.60469,0.60114,0.59214,0.27781,0.36167,0.85521,0.39073,0.97471,0.82394,0.18492,0.19253,0.55573,0.58875,0.54130,0.00000,0.21029,0.00000,0.36282,0.39804,0.02174,0.02174,0.33429,0.59224,0.23364,0.34548,0.49393,0.51963,0.00200,0.00150,-0.00360,-0.00150,0.00370,0.00560,0.00030,-0.00510,-0.03170
2022-05-12,0.51113,0.49906,0.51982,0.07170,0.55418,0.62039,0.68259,0.73259,0.12838,0.85658,0.97379,0.32664,0.31492,0.35184,0.50464,0.00000,0.59048,0.27117,0.60967,0.59271,0.60248,0.63214,0.62006,0.59427,0.58496,0.56800,0.29257,0.59799,0.5922

In [40]:
#train,test 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled_df.drop('close', 1), scaled_df['close'], test_size=0.3, random_state=0, shuffle=False)
    

In [41]:
# tensorflow dataset을 활용한 시퀀스 데이터셋 구성
import tensorflow as tf
def windowed_dataset(series, window_size, batch_size, shuffle):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

In [42]:
# hyper parameter 정의 
WINDOW_SIZE=20
BATCH_SIZE=32

In [43]:
# trian_data는 학습용 데이터셋, test_data는 검증용 데이터셋
train_data = windowed_dataset(y_train, WINDOW_SIZE, BATCH_SIZE, True)
test_data = windowed_dataset(y_test, WINDOW_SIZE, BATCH_SIZE, False)


In [44]:
 # 모델 생성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential([
    # 1차원 feature map 생성
    Conv1D(filters=32, kernel_size=5,
        padding="causal",
        #활성화함수를 relu로 정의
        activation="relu",
        input_shape=[WINDOW_SIZE, 1]),
    # LSTM
    LSTM(16, activation='tanh'),    
    Dense(16, activation="relu"),
    Dense(1)])

loss = Huber()
optimizer = Adam(0.005) ##3compile 시 optimizer는 adam으로 사용
model.compile(loss=Huber(), optimizer=optimizer, metrics=['mse'])

earlystopping = EarlyStopping(monitor='val_loss', patience=10)

filename = os.path.join('tmp', 'ckeckpointer.ckpt')
checkpoint = ModelCheckpoint(filename, 
                            save_weights_only=True,
                            save_best_only=True, 
                            monitor='val_loss',
                            verbose=1)

In [45]:
# 모델 학습
history = model.fit(train_data, 
                validation_data=(test_data), 
                epochs=50,   #50번 학습
                callbacks=[checkpoint, earlystopping])

Epoch 1/50
95/95 [==============================] - 18s 103ms/step - loss: 0.0010 - mse: 0.0021 - val_loss: 9.2101e-04 - val_mse: 0.0018

Epoch 00001: val_loss improved from inf to 0.00092, saving model to tmp\ckeckpointer.ckpt
Epoch 2/50
95/95 [==============================] - 2s 17ms/step - loss: 1.7049e-04 - mse: 3.4097e-04 - val_loss: 5.7456e-04 - val_mse: 0.0011

Epoch 00002: val_loss improved from 0.00092 to 0.00057, saving model to tmp\ckeckpointer.ckpt
Epoch 3/50
95/95 [==============================] - 2s 17ms/step - loss: 1.4327e-04 - mse: 2.8653e-04 - val_loss: 0.0010 - val_mse: 0.0020

Epoch 00003: val_loss did not improve from 0.00057
Epoch 4/50
95/95 [==============================] - 2s 16ms/step - loss: 1.1647e-04 - mse: 2.3294e-04 - val_loss: 3.9535e-04 - val_mse: 7.9070e-04

Epoch 00004: val_loss improved from 0.00057 to 0.00040, saving model to tmp\ckeckpointer.ckpt
Epoch 5/50
95/95 [==============================] - 2s 17ms/step - loss: 1.0866e-04 - mse: 2.1733e-04

In [46]:
pred = model.predict(test_data)

In [47]:
rescaled_y_test = scaler1.inverse_transform(np.array(y_test).reshape(-1, 1))
rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))

In [48]:
# 평가지표 함수
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

def confirm_result(y_test, y_pred):
    MAE = mean_absolute_error(y_test, y_pred)
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    MSLE = mean_squared_log_error(y_test, y_pred)
    RMSLE = np.sqrt(mean_squared_log_error(y_test, y_pred))
    R2 = r2_score(y_test, y_pred)
        
    pd.options.display.float_format = '{:.5f}'.format
    Result = pd.DataFrame(data=[MAE,RMSE, RMSLE, R2],
                            index = ['MAE','RMSE', 'RMSLE', 'R2'],
                            columns=['Results'])
    return Result

In [49]:
confirm_result(rescaled_y_test[20:], rescaled_pred)

,Results
MAE,3267.31106
RMSE,4730.21897
RMSLE,0.04156
R2,0.94634
